In [1]:
import sys
import os
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import json
import yaml
import pandas as pd
import mask_generator.settings as settings

pd.set_option('display.max_columns', None)

In [2]:
EXPERIMENTS_DIR = "../experiments"

def load_yaml(path: str) -> dict:
    with open(path, "r") as f:
        return yaml.safe_load(f)

def load_json(path: str) -> dict:
    with open(path, "r") as f:
        return json.load(f)

def load_run(run_path: str) -> dict:
    config_path = os.path.join(run_path, settings.config_filename)
    results_path = os.path.join(run_path, settings.results_filename)

    if not os.path.exists(config_path) or not os.path.exists(results_path):
        return {}

    config = load_yaml(config_path)
    results = load_json(results_path)

    return {
        "config": config,
        "results": results
    }

def flatten_dict(d: dict, parent_key: str = '', sep: str = '.') -> dict:
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def load_experiments(experiments_dir: str = EXPERIMENTS_DIR) -> pd.DataFrame:
    run_dirs = [os.path.join(experiments_dir, d) for d in os.listdir(experiments_dir)
                if os.path.isdir(os.path.join(experiments_dir, d))]

    data = []
    for run_dir in run_dirs:
        run_data = load_run(run_dir)
        if not run_data:
            continue
        flat_config = flatten_dict(run_data["config"])
        flat_results = flatten_dict(run_data["results"])
        summary = {**flat_config, **flat_results}
        summary["run_dir"] = run_dir
        data.append(summary)

    df = pd.DataFrame(data)
    if df.empty:
        raise ValueError("No valid experiment data found.")
    df.reset_index(drop=True, inplace=True)
    return df

In [3]:
if not os.path.exists(EXPERIMENTS_DIR):
    raise FileNotFoundError(f"Experiments directory '{EXPERIMENTS_DIR}' does not exist.")
df = load_experiments(EXPERIMENTS_DIR)
print(f"df columns: {df.columns.tolist()}")

df.to_csv(os.path.join(EXPERIMENTS_DIR, "experiments_summary.csv"), index=False)

df columns: ['model.arch', 'model.in_channels', 'model.out_channels', 'model.model_args.encoder_name', 'model.model_args.encoder_weights', 'model.model_args.decoder_attention_type', 'model.model_args.activation', 'training.train_dataset_path', 'training.eval_dataset_path', 'training.seed', 'training.batch_size', 'training.num_epochs', 'training.lr', 'training.step_size', 'training.gamma', 'training.patience', 'training.delta', 'training.train_image_size', 'training.augmentations', 'training.use_amp', 'training.qat.enabled', 'training.qat.backend', 'training.loss', 'other.name', 'other.run_hash', 'other.run_dir', 'other.git_commit', 'other.verbose', 'best_epoch', 'elapsed_time_sec', 'elapsed_time_str', 'test_metrics.loss', 'test_metrics.dice', 'test_metrics.iou', 'test_metrics.acc', 'test_metrics.precision', 'test_metrics.recall', 'test_metrics.f1', 'run_dir', 'model.model_args.n_convs', 'model.model_args.filters', 'model.model_args.dropout', 'training.train_dataset', 'training.eval_dat

In [4]:
df1 = df.copy()

# Sort by test_metrics.dice
df1.sort_values(by=["test_metrics.dice"], ascending=False, inplace=True)
# Reset index after sorting
df1.reset_index(drop=True, inplace=True)
# Display the first 10 rows of the sorted DataFrame
print("Top 10 experiments sorted by test_metrics.dice:")
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(df1.head(10))

Top 10 experiments sorted by test_metrics.dice:


,model.arch,model.in_channels,model.out_channels,model.model_args.encoder_name,model.model_args.encoder_weights,model.model_args.decoder_attention_type,model.model_args.activation,training.train_dataset_path,training.eval_dataset_path,training.seed,training.batch_size,training.num_epochs,training.lr,training.step_size,training.gamma,training.patience,training.delta,training.train_image_size,training.augmentations,training.use_amp,training.qat.enabled,training.qat.backend,training.loss,other.name,other.run_hash,other.run_dir,other.git_commit,other.verbose,best_epoch,elapsed_time_sec,elapsed_time_str,test_metrics.loss,test_metrics.dice,test_metrics.iou,test_metrics.acc,test_metrics.precision,test_metrics.recall,test_metrics.f1,run_dir,model.model_args.n_convs,model.model_args.filters,model.model_args.dropout,training.train_dataset,training.eval_dataset
0,my_unet,3,1,NaN,NaN,NaN,NaN,NaN,NaN,42,32,100,0.001,10,0.1,30,0.0,"[256, 256]",NaN,False,False,fbgemm,"[{'name': 'bce', 'weight': 0.3, 'params': {'pos_weight': True}}, {'name': 'dice', 'weight': 0.7, 'params': {'smooth': 1.0}}]",20250626_012301_748adaf2f4,748adaf2f4,/root/MaskGenerator/experiments/20250626_012301_748adaf2f4,59aebd68e885c186bd2aedd2d7a8952ce35f71e7,False,68,12349.954107,03:25:49,0.184593,0.820974,0.696181,0.990053,0.724892,0.946170,0.820880,../experiments/20250626_012301_748adaf2f4,2.0,"[32, 64, 128, 256]",0.0,"[{'csv': '/root/MaskGenerator/Datasets/simu_v0/simu.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}, {'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_val_target.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}]",[{'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_test.csv'}]
1,my_unet,3,1,NaN,NaN,NaN,NaN,NaN,NaN,42,32,100,0.001,10,0.1,30,0.0,"[256, 256]",NaN,False,False,fbgemm,"[{'name': 'bce', 'weight': 0.5, 'params': {'pos_weight': True}}, {'name': 'dice', 'weight': 0.5, 'params': {'smooth': 1.0}}]",20250626_045053_6f4e6a5904,6f4e6a5904,/root/MaskGenerator/experiments/20250626_045053_6f4e6a5904,59aebd68e885c186bd2aedd2d7a8952ce35f71e7,False,38,7757.656906,02:09:17,0.186067,0.820952,0.696028,0.990060,0.725519,0.944822,0.820774,../experiments/20250626_045053_6f4e6a5904,2.0,"[32, 64, 128, 256]",0.0,"[{'csv': '/root/MaskGenerator/Datasets/simu_v0/simu.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}, {'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_val_target.csv', 'augmentations': []}]",[{'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_test.csv'}]
2,my_unet,3,1,NaN,NaN,NaN,NaN,NaN,NaN,42,32,100,0.001,10,0.1,30,0.0,"[256, 256]",NaN,False,False,fbgemm,"[{'name': 'bce', 'weight': 0.5, 'params': {'pos_weight': True}}, {'name': 'dice', 'weight': 0.5, 'params': {'smooth': 1.0}}]",20250625_224145_d2b4a4a135,d2b4a4a135,/root/MaskGenerator/experiments/20250625_224145_d2b4a4a135,59aebd68e885c186bd2aedd2d7a8952ce35f71e7,False,46,9558.466459,02:39:18,0.191770,0.764375,0.616437,0.985536,0.630561,0.964937,0.762711,../experiments/20250625_224145_d2b4a4a135,2.0,"[32, 64, 128, 256]",0.0,"[{'csv': '/root/MaskGenerator/Datasets/simu_v0/simu.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}, {'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_val_target.csv', 'augmentations': ['geometry', 'dropout', 'color_invariance', 'color_variation', 'blur', 'noise', 'weather']}]",[{'csv': '/root/MaskGenerator/Datasets/CARLANE/MoLane/molane_test.csv'}]
3,my_unet,3,1,NaN,NaN,NaN,NaN,/root/MaskGenerator/Datasets/train/simu_v0,/root/MaskGenerator/Datasets/test/MoLane,42,32,100,0.001,10,0.1,30,0.0,"[256, 256]","[geometry, dropout, color_invariance, color_variation, blur, noise, weather]",False,False,fbgemm,"[{'name': 'bce', 'weight': 0.3, 'params': {'pos_weight': True}}, {'name': 'dice